In [1]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [2]:
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.2 MB/s eta 0:00:00


In [3]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
%pip install pydub

In [5]:
import openai
from getpass import getpass
from pydub import AudioSegment
import os
import openai
from openai import OpenAI
import time
import json
from datetime import date
import csv

# set the key discretely as I want to share this notebook and not leak anything confidential
openai.api_key = getpass('Enter your OpenAI API Key: ')

Enter your OpenAI API Key: ··········


Docs are outdated so I need to come up with a newer working version

In [6]:
client = OpenAI(api_key=openai.api_key)

In [7]:
def transcribe_audio(file_path):
    with open(file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file)
    return transcription.text

In [8]:
today = str(date.today())

In [9]:
gpt_model = "gpt-4-1106-preview"

In [10]:
file_path = '/content/audio/AI_Startups_Talk.mp3'

In [11]:
transcription = transcribe_audio(file_path)

In [13]:
print('\n * Transcription *\n')
print(transcription)


 * Transcription *

How would you differentiate between an idea that could be a great foundation for a billion-dollar company and an idea that is likely to get run over by GPT-5? Something that's boring might actually be an incredible business, but why is that? Yeah, let's talk about GPT wrappers. Are people worried about giving these data sets to open AI? All these AI agents are passing the Turing test. I mean, this is why I think the chat interface is wrong. You want to do something in AI, like, this is a good place to, like, look into. Big generational companies are getting built as we speak. Great startup ideas just lying on the ground, you'd, like, trip over them. This might actually be, like, a once-in-a-lifetime opportunity, and I think I actually agree. What a time to be alive! Welcome to the very first episode of The Light Cone. I'm Gary, this is Jared, Harj, and Diana, and we're group partners at Y Combinator, and we get to work with some of the best founders in the world. J

In [22]:
# now I will save the transcription into a .txt file
with open('transcription.txt', 'w') as f:
    f.write(transcription)

Awesome! I've successfully managed to transcribe the AI startups talk using whisper and then save it to a .txt file. Now it's time do some cool analysis.

In [23]:
def abstract_summary_extraction(transcription):
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following transcription of a meeting and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response.choices[0].message.content

In [24]:
abstract = abstract_summary_extraction(transcription)

In [25]:
print('\n * Abstract *\n')
print(abstract)


 * Abstract *

In the inaugural episode of The Light Cone podcast, Y Combinator partners Gary, Jared, Harj, and Diana discuss the burgeoning impact of AI on startups and society. They explore the significance of AI in current startup trends, noting that nearly half of YC's Summer 2023 batch involves large language models (LLMs). The conversation highlights the importance of focusing on specific, often mundane, business problems that AI can address, rather than chasing after broad, shiny applications that may become "tar pits" for founders. They emphasize the potential for AI to automate repetitive tasks, such as government contract bidding, and the opportunity for young founders to enter the AI field due to the level playing field created by the novelty of the technology.

The group also touches on the ethical considerations of AI, the importance of open-source models for democratizing access, and the potential for AI to revolutionize existing software products. They caution against g

In [26]:
def key_points_extraction(transcription):
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    response = response.choices[0].message.content
    return response

In [27]:
key_points = key_points_extraction(transcription)

In [28]:
print('\n * Key Points *\n')
print(key_points)


 * Key Points *

1. The podcast "The Light Cone" is hosted by Y Combinator partners who discuss technology's past and future.
2. AI is increasingly integrated into society and startups, with about 50% of Y Combinator's Summer 2023 batch involving large language models (LLMs).
3. The surge in AI startups is driven by smart founders who see the potential to build large, generational companies.
4. There's a trend of founders dropping out of college to pursue AI opportunities, which are seen as once-in-a-lifetime.
5. AI is creating opportunities for workflow automation by replacing repetitive human tasks, particularly in information processing.
6. There's a perception that Y Combinator prefers AI companies, but in reality, they fund smart founders regardless of their focus area.
7. The podcast discusses the potential for AI to automate mundane tasks, such as searching for government contracts, which can be lucrative despite seeming boring.
8. The term "AI tar pits" refers to startup ideas

In [29]:
def action_item_extraction(transcription):
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    response = response.choices[0].message.content
    return response


In [30]:
action_items = action_item_extraction(transcription)

In [31]:
print('\n * Action Items *\n')
print(action_items)


 * Action Items *

Based on the conversation, here are the action items and tasks that were mentioned:

1. Founders should look into AI applications for mundane information processing jobs, particularly those involving repetitive tasks like searching for things or filling out forms.

2. Founders should consider the potential of AI in automating the search for government contracts and the submission of proposals, as exemplified by the company Sweet Spot.

3. Founders should explore the development of developer tools for prompt engineering, especially tools that can chain together different prompts and test them.

4. Founders should investigate the creation of cybersecurity solutions for large language models (LLMs), such as prompt armor, which prevents the model from regurgitating private data.

5. Founders should consider building AI voice agents for small businesses to act as virtual receptionists.

6. Founders should reimagine existing software with the power of AI, considering how 

In [32]:
def participant_list(transcription):
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting names and roles of the people speaking. Please review the text and identify each person named in the discussions, their title or role, and any other personal information they provide such as location.  Be sure to review the entire conversation and include new people named later in the meeting.  The meeting may be a company earnings conference call with analysts; if this is the case be sure to include the analysts asking questions later in the call.  Please list all of the the names and their related information clearly and concisely.  If there are clear groups of people, such as customer and supplier, group them accordingly"
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    response = response.choices[0].message.content
    return response

In [33]:
participants = participant_list(transcription)

In [34]:
print('\n * Participants *\n')
print(participants)


 * Participants *

The conversation involves four individuals who are group partners at Y Combinator. Their names and roles are as follows:

1. Gary - Group Partner at Y Combinator
2. Jared - Group Partner at Y Combinator
3. Harj - Group Partner at Y Combinator
4. Diana - Group Partner at Y Combinator

No specific personal information such as location is provided for any of the individuals in the conversation. The discussion revolves around the impact of AI on startups, the emergence of AI in various sectors, and the potential for new companies in the field of AI. They also discuss the concept of "GPT wrappers" and the importance of focusing on specific use cases rather than general applications to avoid being outcompeted by more advanced AI models in the future.

Additionally, they mention a company called Sweet Spot, which pivoted to using LLMs for automating the search for government contracts, and another company called Credel, which specializes in customizing AI models to private

In [35]:
def sentiment_analysis(transcription):
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    response = response.choices[0].message.content
    return response

In [36]:
sentiment = sentiment_analysis(transcription)

In [37]:
print('\n * Sentiment *\n')
print(sentiment)


 * Sentiment *

The sentiment of the text is generally positive, with a strong sense of excitement and optimism about the potential of AI and startup opportunities. The speakers convey enthusiasm about the transformative impact of AI on various industries and the emergence of new companies. They discuss the potential for once-in-a-lifetime opportunities and the sense that great startup ideas are abundant and accessible. The tone is one of encouragement for founders to explore AI and capitalize on the current technological advancements.

The language used includes phrases like "what a time to be alive," "incredible business," "once-in-a-lifetime opportunity," and "big generational companies are getting built," which all contribute to a positive sentiment. The discussion also touches on the challenges and considerations in the AI space, such as data privacy and the risk of "tar pit" ideas, but these are framed as part of the exciting landscape rather than as deterrents.

Overall, the se

Great! I've covered different use cases of how to retrieve and generate helpful information relating to longer form content such as a meeting/podcast/lecture.